# S&P 500

## Library import

In [25]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Import list of Stocks

In [26]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks.head(10)

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
5,ABC
6,ABMD
7,ABT
8,ACN
9,ADBE


## Acquiring an API Token

In [27]:
from secrets import IEX_CLOUD_API_TOKEN

## Making first API call
* Market capitalization for each stock
* Price of each stock

In [28]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
# print(data)

## Parsing API call

In [29]:
price = data['latestPrice']
market_cap = data['marketCap']

## Adding stocks data to a Pandas DataFrame

In [30]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns=my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [31]:
final_dataframe = final_dataframe.append(pd.Series(
    [
        symbol,
        price,
        market_cap,
        'N/A'
    ],
        index=my_columns),
    ignore_index=True
)

final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,178.99,2926245810110,N/A


## Looping trough the tickers in the list of stocks (very slow)

In [32]:
final_dataframe = pd.DataFrame(columns=my_columns)
for stock in stocks['Ticker']:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
        pd.Series(
        [
            stock,
            data['latestPrice'],
            data['marketCap'],
            'N/A'
        ],
        index=my_columns),
    ignore_index=True
    )

In [33]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,137.65,41015409139,N/A
1,AAL,17.93,11356005634,N/A
2,AAP,220.54,13088153636,N/A
3,AAPL,173.81,2809335350034,N/A
4,ABBV,172.47,304441955529,N/A
5,ABC,158.54,34162189592,N/A
6,ABMD,324.69,14927875493,N/A
7,ABT,121.65,215871090040,N/A
8,ACN,356.51,231623800790,N/A
9,ADBE,457.62,218502745096,N/A


## Using Batch API calls to improve performance 

In [34]:
def chunks(lst, n):
    """ Yield succesive n-sized chunks from lst"""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [35]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    # print(symbol_strings[i])

final_dataframe = pd.DataFrame(columns= my_columns)

for symbol_string in symbol_strings:
    batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url).json()
    
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(pd.Series(
            [
              symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['marketCap'],
                'N/A'
            ],
            index=my_columns),
        ignore_index=True
        )

final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,133.63,41301706773,N/A
1,AAL,17.39,11302402317,N/A
2,AAP,213.24,13510562501,N/A
3,AAPL,176.25,2852673723096,N/A
4,ABBV,176.14,308995587242,N/A
5,ABC,161.51,33539804737,N/A
6,ABMD,320.10,14654340224,N/A
7,ABT,120.07,220019553892,N/A
8,ACN,354.15,232453809944,N/A
9,ADBE,446.08,218726435361,N/A


In [36]:
final_dataframe.shape

(501, 4)

## Calculating the number of shares to buy

In [37]:
while True:
    try:
        portfolio_size = float(input('Enter the value of your portfolio: '))
        break
    except ValueError:
                print("Thats not  number! \nPlease try again")

Enter the value of your portfolio: 100000000


In [38]:
position_size = portfolio_size / len(final_dataframe.index)
position_size

199600.79840319362

In [39]:
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Stock Price'])
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,133.63,41301706773,1493
1,AAL,17.39,11302402317,11477
2,AAP,213.24,13510562501,936
3,AAPL,176.25,2852673723096,1132
4,ABBV,176.14,308995587242,1133
5,ABC,161.51,33539804737,1235
6,ABMD,320.10,14654340224,623
7,ABT,120.07,220019553892,1662
8,ACN,354.15,232453809944,563
9,ADBE,446.08,218726435361,447


## Formating Excel Output

### Initializing XlsxWriter Object

In [40]:
writer = pd.ExcelWriter('Recommended_Trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, 'Recommended Trades', index=False)

### Creating the formats needed for the .xlsx file

In [41]:
background_color = '#0a023'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

### Applying the Formats to the Columns of the .xlsx file

In [42]:
# writer.sheets['Recommended Trades'].set_column('A:A', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('B:B', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('C:C', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('D:D', 18, string_format)
# writer.save()

writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
writer.sheets['Recommended Trades'].write('B1', 'Stock Price', dollar_format)
writer.sheets['Recommended Trades'].write('C1', 'Market Capitalization', dollar_format)
writer.sheets['Recommended Trades'].write('D1', 'Number of Shares to Buy', integer_format)

0

In [43]:
column_formats = {
    'A':['Ticker', string_format],
    'B':['Stock Price', dollar_format],
    'C':['Market Capitalization', dollar_format],
    'D':['Number of Shares to Buy', integer_format]
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

### Save the Excel Output

In [44]:
writer.save()